In [1]:
from pathlib import Path
import cv2
from PIL import Image

from core.data.services.default_video_service import DefaultVideoService as VideoService
from core.data.services.default_homography_service import DefaultHomographyService as HomographyService
from core.data.services.default_finder_service import DefaultFinderService as FinderService
from core.infra.scene_objects.corners import Corners
from core.infra.scene_objects.offset import Offset
from core.infra.context.video_writer_context import VideoWriterContext

/Users/yann/Documents/Projects/Python/basket-tracking/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
assets = Path().cwd().joinpath("assets")
input_video_path = assets.joinpath("videosample_10s.mp4")
minimap_image_path = assets.joinpath("minimap.jpeg")
minimap_video_output_path = Path().cwd().joinpath("output").joinpath("output.mp4")

In [3]:
minimap_image = cv2.imread(str(minimap_image_path))

In [4]:
region_of_interest = Corners.from_rectangle(top_left_x=0, top_left_y=750, bottom_right_x=3840, bottom_right_y=1550)

In [5]:
video_service = VideoService()
homography_service = HomographyService()
finder_service = FinderService()

In [6]:
minimap_corners = Corners(
    bottom_right=Offset(x=40, y=420),
    bottom_left=Offset(x=40, y=42),
    top_left=Offset(x=752, y=42),
    top_right=Offset(x=752, y=420)
)

In [7]:
scenes = video_service.getScenesFromVideo(video_path=input_video_path, region_of_interest=region_of_interest)

In [11]:
with VideoWriterContext(path=minimap_video_output_path) as context:
    for court_scene, court_frame in scenes:
        court_corners = finder_service.findCourt(court_frame).corners
        minimap_scene = homography_service.applyHomography(scene=court_scene, from_corners=court_corners, to_corners=minimap_corners)

        frame = minimap_scene.draw_frame(frame=minimap_image)
        
        context.write(frame=frame)


NameError: name 'VideoWriterContext' is not defined

In [10]:


court_scene, court_frame = next(scenes)
court_corners = finder_service.findCourt(court_frame).corners
minimap_scene = homography_service.applyHomography(scene=court_scene, from_corners=court_corners, to_corners=minimap_corners)

frame1 = minimap_scene.draw_frame(frame=minimap_image)
frame2 = court_scene.draw_frame(frame=court_frame)
Image.fromarray(frame1[:, :, ::-1]).show()
Image.fromarray(frame2[:, :, ::-1]).show()
